In [1]:
# STEP 1 — Load Merged Base (Ensure correct file name/path)
import pandas as pd
import numpy as np
import os

# Assuming the file path is correct relative to the notebook
# Adjust if the file name is 'merged_application.parquet'
try:
    df = pd.read_csv("../outputs/merged_base_final.csv")
except FileNotFoundError:
    print("Trying to load parquet file instead...")
    df = pd.read_parquet("../outputs/merged_application.parquet")
    
print(f"✅ Data loaded. Shape: {df.shape}")
df.head()

Trying to load parquet file instead...
✅ Data loaded. Shape: (307511, 368)


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,cc__SK_DPD__max,cc__SK_DPD__min,cc__SK_DPD__sum,cc__SK_DPD__std,cc__SK_DPD_DEF__count,cc__SK_DPD_DEF__mean,cc__SK_DPD_DEF__max,cc__SK_DPD_DEF__min,cc__SK_DPD_DEF__sum,cc__SK_DPD_DEF__std
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0


In [2]:
# Cell 2: Handle Missing Values (Domain Logic)
# Income missing → assume low/stable reporting
df['AMT_INCOME_TOTAL'].fillna(df['AMT_INCOME_TOTAL'].median(), inplace=True)

# Age / employment missing → capped (365243 is a placeholder for unknown status)
df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)
df['DAYS_EMPLOYED'].fillna(df['DAYS_EMPLOYED'].median(), inplace=True)

# Note: Credit card features and other aggregated features are handled later in Part B/C/D/E logic or were imputed in Notebook 1.

print("✅ STEP 2: Core Missing Values Handled (Income, Employment)")

✅ STEP 2: Core Missing Values Handled (Income, Employment)


C:\Users\Asus\AppData\Local\Temp\ipykernel_10636\3278163030.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['AMT_INCOME_TOTAL'].fillna(df['AMT_INCOME_TOTAL'].median(), inplace=True)
C:\Users\Asus\AppData\Local\Temp\ipykernel_10636\3278163030.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always b

In [3]:
# Cell 3: PART-A — Create Core Applicant Features

# 1. Age (positive value in years)
df['AGE'] = (-df['DAYS_BIRTH']) / 365

# 2. Employment length (positive value in years)
df['EMPLOYMENT_YEARS'] = (-df['DAYS_EMPLOYED']) / 365

# 3. Income-to-Credit Ratio (Ability to handle large debt vs income)
df['INCOME_CREDIT_RATIO'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']

# 4. Annuity-to-Income Ratio (Debt Service Ratio - DSR) (Percentage of income spent on this EMI)
df['ANNUITY_INCOME_RATIO'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']

# 5. Credit Term (Loan duration in years/months - often a strong predictor)
df['CREDIT_TERM'] = df['AMT_CREDIT'] / df['AMT_ANNUITY']

print("STEP 3: Core Applicant Features created.")

STEP 3: Core Applicant Features created.


In [4]:
# Cell 4: PART-B — STEP 4: Payment Ratio

# NOTE: The ratio in the original prompt (AMT_INSTALMENT / AMT_PAYMENT) is inverted for risk logic.
# A standard Payment Ratio is (Paid / Expected). Let's use the features created in Notebook 1
# which should be aggregated sum/mean, not the base columns.

# Assuming these columns are the aggregated sums from Notebook 1 (installments_payments):
# If your Notebook 1 aggregation used different names, you must correct them.

# Payment Paid vs Expected (Aggregated Sums)
df['PAYMENT_RATIO'] = df['inst__AMT_PAYMENT__sum'] / df['inst__AMT_INSTALMENT__sum']

# Handle division by zero/inf (Set extreme values to NaN)
df['PAYMENT_RATIO'] = df['PAYMENT_RATIO'].replace([np.inf, -np.inf], np.nan) 
# Clip very high values for stability (e.g., max 3.0)
df['PAYMENT_RATIO'] = df['PAYMENT_RATIO'].clip(upper=3.0) 

print(" STEP 4: Payment Ratio created.")

 STEP 4: Payment Ratio created.


In [6]:
# Cell 5: PART-B — STEP 5: Days Late Payment (Revised Fix)

# We cannot rely on inst__DAYS_ENTRY_PAYMENT__mean and inst__DAYS_INSTALMENT__mean.

# Option 1: Try to create DAYS_LATE using the DPD max value aggregated in Notebook 1
try:
    # Assuming 'inst__DPD__max' or 'inst__AMT_DELINQUENCY__max' exists
    # We will look for an explicit DPD column from the installments table
    df['DAYS_LATE'] = df['inst__DPD__max'] 
    
except KeyError:
    # Option 2: Use the Max DPD from the Bureau Credit History (often named DPD__max)
    try:
        df['DAYS_LATE'] = df['bureau__DAYS_CREDIT_ENDDATE__max'] - df['bureau__DAYS_CREDIT__max']
    except KeyError:
        # Final Fallback: Set to a safe, low default value (only if no DPD feature exists)
        df['DAYS_LATE'] = 0 
        print(" FINAL FALLBACK: No DPD feature found. DAYS_LATE set to 0. Check Notebook 1 Aggregation for DPD features.")


# Ensure DAYS_LATE is positive (we only care about late days, not early days)
df['DAYS_LATE'] = df['DAYS_LATE'].apply(lambda x: x if x > 0 else 0) 

# Fix: Use direct assignment instead of inplace=True (to fix FutureWarning)
df['DAYS_LATE'] = df['DAYS_LATE'].fillna(0) 

print("✅ STEP 5: Days Late Payment feature created (Check for final fallback warning).")

✅ STEP 5: Days Late Payment feature created (Check for final fallback warning).


In [7]:
# Cell 6: PART-C — Previous Application Features

# NOTE: Since these features are aggregations (mean, count) performed on the 'previous_application' table
# and merged back to the main dataframe, we must use the correct aggregated column names
# (which typically start with a prefix like 'prev__'). 
# If a KeyError occurs, check your Notebook 1 aggregation names.

try:
    # 1. Total previous loans: Count of all previous applications for the current customer
    # We assume 'prev__SK_ID_PREV__count' was the aggregation used to count previous applications.
    df['PREV_LOANS_COUNT'] = df['prev__SK_ID_PREV__count']
except KeyError:
    print(" Warning: 'prev__SK_ID_PREV__count' not found. Skipping feature PREV_LOANS_COUNT.")
    df['PREV_LOANS_COUNT'] = 0 # Fallback

try:
    # 2. Approved vs Rejected history: Mean of the approval flag (Approval=1, Reject=0)
    # Mean of a binary flag gives the approval rate.
    df['PAST_APPROVAL_RATE'] = df['prev__PREV_APPROVED__mean']
except KeyError:
    print(" Warning: 'prev__PREV_APPROVED__mean' not found. Skipping feature PAST_APPROVAL_RATE.")
    df['PAST_APPROVAL_RATE'] = np.nan # Fallback

try:
    # 3. Past credit amount behaviour: Average credit requested in previous applications
    df['PREV_AVG_CREDIT'] = df['prev__AMT_CREDIT_prev__mean']
except KeyError:
    print(" Warning: 'prev__AMT_CREDIT_prev__mean' not found. Skipping feature PREV_AVG_CREDIT.")
    df['PREV_AVG_CREDIT'] = np.nan # Fallback


# Fill missing values (for customers with no previous applications) with 0 or mean
df['PREV_LOANS_COUNT'] = df['PREV_LOANS_COUNT'].fillna(0)
df['PAST_APPROVAL_RATE'] = df['PAST_APPROVAL_RATE'].fillna(df['PAST_APPROVAL_RATE'].mean())
df['PREV_AVG_CREDIT'] = df['PREV_AVG_CREDIT'].fillna(0)


print("STEP 6: Previous Application Features created.")

STEP 6: Previous Application Features created.


In [8]:
# Cell 7: PART-D — Bureau Features

# NOTE: These features depend on the aggregation columns created from the 'bureau' and 'bureau_balance' tables in Notebook 1.
# If KeyErrors occur, you must check your Notebook 1 aggregation names (e.g., 'bureau__...').

try:
    # 1. Number of bureau loans: Total count of bureau loans for the current customer
    df['BUREAU_LOAN_COUNT'] = df['bureau__SK_ID_BUREAU__count'] 
except KeyError:
    print(" Warning: 'bureau__SK_ID_BUREAU__count' not found. Skipping feature BUREAU_LOAN_COUNT.")
    df['BUREAU_LOAN_COUNT'] = 0 # Fallback

try:
    # 2. Active loans: Sum of the flag indicating active credit (1=Active, 0=Closed)
    df['ACTIVE_LOANS'] = df['bureau__CREDIT_ACTIVE_flag__sum']
except KeyError:
    print(" Warning: 'bureau__CREDIT_ACTIVE_flag__sum' not found. Skipping feature ACTIVE_LOANS.")
    df['ACTIVE_LOANS'] = 0 # Fallback

try:
    # 3. Overdue behaviour: Total sum of the overdue amount reported by the bureau
    df['BUREAU_OVERDUE_SUM'] = df['bureau__AMT_CREDIT_SUM_OVERDUE__sum']
except KeyError:
    print(" Warning: 'bureau__AMT_CREDIT_SUM_OVERDUE__sum' not found. Skipping feature BUREAU_OVERDUE_SUM.")
    df['BUREAU_OVERDUE_SUM'] = 0 # Fallback

try:
    # 4. Average credit days overdue (DPD): Mean DPD over the customer's bureau history
    df['BUREAU_DPD'] = df['bureau__DPD__mean'] # Assuming a DPD mean was calculated
except KeyError:
    print(" Warning: 'bureau__DPD__mean' not found. Skipping feature BUREAU_DPD.")
    df['BUREAU_DPD'] = 0 # Fallback

# Fill missing values (for customers with no bureau history) with 0
df['BUREAU_LOAN_COUNT'] = df['BUREAU_LOAN_COUNT'].fillna(0)
df['ACTIVE_LOANS'] = df['ACTIVE_LOANS'].fillna(0)
df['BUREAU_OVERDUE_SUM'] = df['BUREAU_OVERDUE_SUM'].fillna(0)
df['BUREAU_DPD'] = df['BUREAU_DPD'].fillna(0)


print(" STEP 7: Bureau Features created.")

 STEP 7: Bureau Features created.


In [10]:
# Cell 8: PART-E — Credit Card Balance Features

# NOTE: These features depend on the aggregation columns created from the 'credit_card_balance' table in Notebook 1.
# The prefix should be 'cc__' (as seen in the merge output).

try:
    # 1. Utilization Ratio: Mean balance divided by the mean credit limit (Measures how much limit they use)
    df['CREDIT_UTILIZATION'] = df['cc__AMT_BALANCE__mean'] / df['cc__AMT_CREDIT_LIMIT_ACTUAL__mean']
except KeyError:
    print(" Warning: Credit card mean balance/limit features not found. Skipping CREDIT_UTILIZATION.")
    df['CREDIT_UTILIZATION'] = np.nan # Fallback

try:
    # 2. Total number of statements: Total count of monthly statements available
    # High count means longer credit history with the card.
    df['CC_MONTHS'] = df['cc__MONTHS_BALANCE__count']
except KeyError:
    print(" Warning: Credit card balance count feature not found. Skipping CC_MONTHS.")
    df['CC_MONTHS'] = np.nan # Fallback

try:
    # 3. Over-limit flag (Mean): Flag indicating if the customer went over their credit limit at any point (Mean > 0 means it happened)
    # This checks if the aggregated column that flags over-limit status exists.
    df['OVER_LIMIT'] = df['cc__OVER_LIMIT_FLAG__mean'].astype(int)
except KeyError:
    # A simplified flag based on the engineered feature from the prompt (if base columns exist)
    try:
        # Recreating the flag based on mean values if the specific aggregated flag is missing:
        df['OVER_LIMIT'] = (df['cc__AMT_BALANCE__mean'] > df['cc__AMT_CREDIT_LIMIT_ACTUAL__mean']).astype(int)
    except KeyError:
        print(" Warning: All credit card balance features are missing. Skipping OVER_LIMIT.")
        df['OVER_LIMIT'] = 0 # Fallback


# Fill missing values (for customers with no credit card history)
df['CREDIT_UTILIZATION'] = df['CREDIT_UTILIZATION'].fillna(0) # Assume 0 utilization if no card history
df['CC_MONTHS'] = df['CC_MONTHS'].fillna(0)
df['OVER_LIMIT'] = df['OVER_LIMIT'].fillna(0)


print(" STEP 8: Credit Card Balance Features created.")

 STEP 8: Credit Card Balance Features created.


In [11]:
# Cell 9: PART-F — Risk Flags

# 1. Young + high credit → risk
# Customers who are young (Age < 25) and have a large loan (AMT_CREDIT > 200,000) are typically seen as high-risk.
df['FLAG_YOUNG_RISK'] = ((df['AGE'] < 25) & (df['AMT_CREDIT'] > 200000)).astype(int)

# 2. Income stress indicator:
# A low Income-to-Credit Ratio (e.g., < 0.2) suggests the loan amount is very high relative to income.
df['FLAG_INCOME_STRESS'] = (df['INCOME_CREDIT_RATIO'] < 0.2).astype(int)

# 3. Payment discipline flag:
# Flagging severe delinquency (e.g., DAYS_LATE > 30 days) is a critical risk indicator.
# We use the DAYS_LATE feature created in PART-B.
df['FLAG_BAD_PAYMENT'] = (df['DAYS_LATE'] > 30).astype(int)


print(" STEP 9: Bank-Style Risk Flags created.")

 STEP 9: Bank-Style Risk Flags created.


In [15]:
# Cell 10: PART-G — ENCODING (Revised)
from sklearn.preprocessing import LabelEncoder
import numpy as np

cat_cols = df.select_dtypes(include=['object']).columns
le = LabelEncoder()

print(f"Total Categorical Columns to re-encode: {len(cat_cols)}")

for col in cat_cols:
    # 1. Convert to string to handle NaNs as a distinct category
    df[col] = df[col].astype(str)
    
    # 2. Apply Label Encoding (results in an integer array)
    df[col] = le.fit_transform(df[col])
    
    # 3. IMPORTANT: Explicitly cast the result to ensure dtype is integer.
    df[col] = df[col].astype('int32')

print(f"Remaining object columns: {len(df.select_dtypes(include=['object']).columns)}")


Total Categorical Columns to re-encode: 0
Remaining object columns: 0


In [13]:
# Cell 11: PART-H — OUTLIER HANDLING (WINSORIZATION)
import numpy as np

print(f"Original shape: {df.shape}")

# Iterate through all numeric columns (float and int)
for col in df.select_dtypes(include=[np.number]).columns:
    
    # Calculate 1st and 99th percentiles
    lower_bound = df[col].quantile(0.01)
    upper_bound = df[col].quantile(0.99)
    
    # Apply clipping (Winsorization)
    df[col] = df[col].clip(lower_bound, upper_bound)

print(" STEP 11: Winsorization (1%-99%) applied to all numeric features.")

Original shape: (307511, 388)
 STEP 11: Winsorization (1%-99%) applied to all numeric features.


In [16]:
# Cell 12: PART-I — FINAL FEATURESET SAVE

# 1. Define the output path (Assuming an 'outputs' folder exists, based on previous file structure)
import os
OUT_DIR = "../outputs/"
os.makedirs(OUT_DIR, exist_ok=True) # Ensure the output directory exists

# 2. Save the final engineered dataframe to CSV
final_path = os.path.join(OUT_DIR, "final_features.csv")
df.to_csv(final_path, index=False)

# 3. Print the final shape and confirmation
print(f"STEP 12: Final features saved successfully to: {final_path}")
print(f"Final Dataframe Shape (Ready for Modeling): {df.shape}")

STEP 12: Final features saved successfully to: ../outputs/final_features.csv
Final Dataframe Shape (Ready for Modeling): (307511, 388)
